# Generar archivos a partir de base de preguntas

### Algunas variables y pasos previos

In [1]:
# archivos a generar en este notebook
gen_etiquetas = True
gen_sinonimos = False
gen_arbol = True
gen_preguntas = True

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import pandas as pd
df1 = pd.read_csv('Etiquetas_diabetes.tsv', sep='\t')
df1.sample(5)

,2q,Etiqueta 1,Etiqueta 2,Etiqueta 3,Resumen pregunta,Link etiqueta 2,keyword1,keyword2,keyword3,keyword4,keyword5,Pregunta,Respuesta,Fuente
421,326,Diabetes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,¿Qué medicamentos son buenos para contrarresta...,NaN,NaN
251,253,Diabetes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uso insulina lenta y rápida; ¿como realizó la....,NaN,NaN
340,326,Diabetes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mi madre padece de diabetes y últimamente me h...,NaN,NaN
55,115,Diabetes,FARMACOS,Insulina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,que hago para la resistencia a la insulina,"Bajar de Peso,Ejercicio físico Regular para ma...",http://www.nhs.uk/
241,243,Diabetes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,el exceso de insulina hace engordar,NaN,NaN


In [3]:
# encontrar numero de columnas keyword y etiqueta
max_keys = 0
max_labels = 0
while True:
    try:
        max_keys += 1
        df1['keyword'+str(max_keys)]
    except:
        max_keys -= 1
        break
        
while True:
    try:
        max_labels += 1
        df1['Etiqueta '+str(max_labels)]
    except:
        max_labels -= 1
        break
print("Cantidad de columnas encontradas. Keyword: {}. Etiquetas: {}".format(max_keys, max_labels))

Cantidad de columnas encontradas. Keyword: 5. Etiquetas: 3


### 1. Pares de keywords y etiquetas

In [4]:
df1.head()

,2q,Etiqueta 1,Etiqueta 2,Etiqueta 3,Resumen pregunta,Link etiqueta 2,keyword1,keyword2,keyword3,keyword4,keyword5,Pregunta,Respuesta,Fuente
0,129,Diabetes,Complicaciones diabetes,Glaucoma,NaN,http://www.diabetes.org/es/vivir-con-diabetes/...,glaucoma,NaN,NaN,NaN,NaN,Quisiera saber acerca de glaucoma y diabetes.,El glaucoma es un grupo de enfermedades que pu...,nei.nih.gov
1,143,Diabetes,Cardiovascular,NaN,NaN,NaN,ictus,NaN,NaN,NaN,NaN,¿los diabeticos tienes más posibilidades de te...,Los pacientes con diabetes tienen mayor riesgo...,fundacioictus.com
2,137,Diabetes,Cirugia y Diabetes,NaN,¿En qué consiste la cirugía para la diabetes?,NaN,cirugía,NaN,NaN,NaN,NaN,¿En qué consiste la cirugía para la diabetes?,Consiste en la cirugía de bypass gástrico Redu...,diabetes.org
3,138,Diabetes,Cirugia y Diabetes,NaN,¿La Cirugía Bariatrica me mejorara la diabetes?,NaN,reducción de estómago,NaN,NaN,NaN,NaN,¿Una persona que decida realizarse una operaci...,A veces se considera a la cirugía bariátrica c...,diabetes.org
4,1,Diabetes,Informacion General y Diabetes,NaN,NaN,NaN,diabetes,NaN,NaN,NaN,NaN,¿Qué es la Diabetes?,Es una Enfermedad crónica que se produce cuand...,idf.org


In [5]:
pares = []
for index, row in df1.iterrows():
    # recorrer cada columna keyword
    for k in range(1, max_keys+1):
        keyword = row['keyword'+str(k)]
        if type(keyword) == str:
            for lab_idx in range(1, max_labels+1):
                label = row['Etiqueta '+str(lab_idx)]
                if type(label) == str:
                    pares.append([label, keyword])
print pares[0:10]

[['Diabetes', 'glaucoma'], ['Complicaciones diabetes', 'glaucoma'], ['Glaucoma', 'glaucoma'], ['Diabetes', 'ictus'], ['Cardiovascular', 'ictus'], ['Diabetes', 'cirug\xc3\xada'], ['Cirugia y Diabetes', 'cirug\xc3\xada'], ['Diabetes', 'reducci\xc3\xb3n de est\xc3\xb3mago'], ['Cirugia y Diabetes', 'reducci\xc3\xb3n de est\xc3\xb3mago'], ['Diabetes', 'diabetes']]


In [6]:
import codecs, json
data = {
    "diabetes": pares
}
if gen_etiquetas:
    with open('../sharedFiles/etiquetas_db.json', 'w') as f:
        json.dump(data, f, ensure_ascii=False, sort_keys = True)#, indent = 4)
else:
    print("No se guarda archivo de etiquetas")

### 2. Sinónimos con w2v

In [7]:
df1.head()

,2q,Etiqueta 1,Etiqueta 2,Etiqueta 3,Resumen pregunta,Link etiqueta 2,keyword1,keyword2,keyword3,keyword4,keyword5,Pregunta,Respuesta,Fuente
0,129,Diabetes,Complicaciones diabetes,Glaucoma,NaN,http://www.diabetes.org/es/vivir-con-diabetes/...,glaucoma,NaN,NaN,NaN,NaN,Quisiera saber acerca de glaucoma y diabetes.,El glaucoma es un grupo de enfermedades que pu...,nei.nih.gov
1,143,Diabetes,Cardiovascular,NaN,NaN,NaN,ictus,NaN,NaN,NaN,NaN,¿los diabeticos tienes más posibilidades de te...,Los pacientes con diabetes tienen mayor riesgo...,fundacioictus.com
2,137,Diabetes,Cirugia y Diabetes,NaN,¿En qué consiste la cirugía para la diabetes?,NaN,cirugía,NaN,NaN,NaN,NaN,¿En qué consiste la cirugía para la diabetes?,Consiste en la cirugía de bypass gástrico Redu...,diabetes.org
3,138,Diabetes,Cirugia y Diabetes,NaN,¿La Cirugía Bariatrica me mejorara la diabetes?,NaN,reducción de estómago,NaN,NaN,NaN,NaN,¿Una persona que decida realizarse una operaci...,A veces se considera a la cirugía bariátrica c...,diabetes.org
4,1,Diabetes,Informacion General y Diabetes,NaN,NaN,NaN,diabetes,NaN,NaN,NaN,NaN,¿Qué es la Diabetes?,Es una Enfermedad crónica que se produce cuand...,idf.org


In [8]:
if gen_sinonimos:
    from gensim import models

    w2v_path = '/media/Tera/Datasets/NLP/sbw_vectors.bin'
    w = models.KeyedVectors.load_word2vec_format(w2v_path, binary=True)
    print("Datos word2vec cargados")

In [9]:
if gen_sinonimos:
    # numero de columnas keyword ya encontrado antes
    print("Cantidad de columnas con keywords_df: {}\n".format(max_keys))
    # guardar todos los keyword
    keywords = []
    for index, row in df1.iterrows():
        for k in range(1, max_keys+1):
            keyword = row['keyword'+str(k)]
            if type(keyword) == str and keyword not in keywords:
                keywords.append(keyword)
    print keywords

In [10]:
if gen_sinonimos:
    sinonimos = []
    for keyword in keywords:
        knn = [keyword]
        try:
            for x in w.most_similar(keyword.decode('utf-8'), topn=5):
                knn.append(x[0])
        except:
            print("{} no existe en vocabulario".format(keyword))
        sinonimos.append(knn)
    sinonimos = pd.DataFrame(sinonimos)
    sinonimos.sample(5)

In [11]:
# maybe output
if gen_sinonimos:
    sinonimos.to_csv('../sharedFiles/sinonimos_w2v_parciales.csv', encoding='utf-8', header=None, index=None)
else:
    print("No se guarda archivo de sinonimos")

No se guarda archivo de sinonimos


### 3. Árbol de etiquetas 

In [12]:
df1.head()

,2q,Etiqueta 1,Etiqueta 2,Etiqueta 3,Resumen pregunta,Link etiqueta 2,keyword1,keyword2,keyword3,keyword4,keyword5,Pregunta,Respuesta,Fuente
0,129,Diabetes,Complicaciones diabetes,Glaucoma,NaN,http://www.diabetes.org/es/vivir-con-diabetes/...,glaucoma,NaN,NaN,NaN,NaN,Quisiera saber acerca de glaucoma y diabetes.,El glaucoma es un grupo de enfermedades que pu...,nei.nih.gov
1,143,Diabetes,Cardiovascular,NaN,NaN,NaN,ictus,NaN,NaN,NaN,NaN,¿los diabeticos tienes más posibilidades de te...,Los pacientes con diabetes tienen mayor riesgo...,fundacioictus.com
2,137,Diabetes,Cirugia y Diabetes,NaN,¿En qué consiste la cirugía para la diabetes?,NaN,cirugía,NaN,NaN,NaN,NaN,¿En qué consiste la cirugía para la diabetes?,Consiste en la cirugía de bypass gástrico Redu...,diabetes.org
3,138,Diabetes,Cirugia y Diabetes,NaN,¿La Cirugía Bariatrica me mejorara la diabetes?,NaN,reducción de estómago,NaN,NaN,NaN,NaN,¿Una persona que decida realizarse una operaci...,A veces se considera a la cirugía bariátrica c...,diabetes.org
4,1,Diabetes,Informacion General y Diabetes,NaN,NaN,NaN,diabetes,NaN,NaN,NaN,NaN,¿Qué es la Diabetes?,Es una Enfermedad crónica que se produce cuand...,idf.org


In [13]:
import unicodedata

# para quitar acentos y caracteres especiales
def remove_accents(input_str):
    input_str = str(input_str)
    input_str = unicode(input_str, 'utf-8')
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

# SE QUITAN LOS ACENTOS Y Ñ, SE APLICA CAPITALIZE
paths = []
for i, row in df1.iterrows():
    path = []
    for j in range(1, max_labels+1):
        # seguir solo si la celda contiene algo
        label = row['Etiqueta '+str(j)]
        if type(label) == str:
            path.append(remove_accents(label).capitalize())
        else:
            continue
    if path not in paths:
        paths.append(path)

# pasar a DataFrame
paths = pd.DataFrame(paths)

# maybe output
if gen_arbol:
    paths.to_csv('../sharedFiles/arbol_etiquetas.csv', encoding='utf-8', header=None, index=None)
else:
    print("No se guarda archivo de sinonimos")

### 4. Etiquetas, preguntas simplificadas y respuestas

In [14]:
columns_labels = ['Pregunta', 'Respuesta', 'Resumen pregunta']
for i in range(1, max_labels+1):
    columns_labels.append('Etiqueta '+str(i))
print columns_labels

preguntas_df = df1[columns_labels].copy()
preguntas_df.head()

['Pregunta', 'Respuesta', 'Resumen pregunta', 'Etiqueta 1', 'Etiqueta 2', 'Etiqueta 3']


,Pregunta,Respuesta,Resumen pregunta,Etiqueta 1,Etiqueta 2,Etiqueta 3
0,Quisiera saber acerca de glaucoma y diabetes.,El glaucoma es un grupo de enfermedades que pu...,NaN,Diabetes,Complicaciones diabetes,Glaucoma
1,¿los diabeticos tienes más posibilidades de te...,Los pacientes con diabetes tienen mayor riesgo...,NaN,Diabetes,Cardiovascular,NaN
2,¿En qué consiste la cirugía para la diabetes?,Consiste en la cirugía de bypass gástrico Redu...,¿En qué consiste la cirugía para la diabetes?,Diabetes,Cirugia y Diabetes,NaN
3,¿Una persona que decida realizarse una operaci...,A veces se considera a la cirugía bariátrica c...,¿La Cirugía Bariatrica me mejorara la diabetes?,Diabetes,Cirugia y Diabetes,NaN
4,¿Qué es la Diabetes?,Es una Enfermedad crónica que se produce cuand...,NaN,Diabetes,Informacion General y Diabetes,NaN


In [15]:
# maybe output
if gen_preguntas:
    preguntas_df.to_csv('../sharedFiles/resumen_preguntas_etiquetas.tsv', sep='\t', encoding='utf-8', index=None)
    print("Todo terminado")
else:
    print("No se guarda archivo de preguntas resumen")

print("\nArchivos generados:\n\t1. Etiquetas:\t{} \n\t2. Sinónimos:\t{} \n\t3. Árbol:\t{} \n\t4. Preguntas:\t{}"
     .format(gen_etiquetas, gen_sinonimos, gen_arbol, gen_preguntas))

Todo terminado

Archivos generados:
	1. Etiquetas:	True 
	2. Sinónimos:	False 
	3. Árbol:	True 
	4. Preguntas:	True
